In [ ]:
import os
# import ipympl
from pathlib import Path
from pdb import set_trace
import pickle
import xarray as xr
import matplotlib.pyplot as plt
from datetime import date, datetime


from gfatpy.lidar.quality_assurance.rayleigh_fit import rayleigh_fit_from_date
from gfatpy.lidar.quality_assurance.plot import plot_rayleigh_fit
from gfatpy.lidar.preprocessing.lidar_preprocessing import preprocess
from gfatpy.lidar.utils.utils import signal_to_rcs
from gfatpy.utils.plot import color_list
%load_ext autoreload
%autoreload 2

%matplotlib inline
# %matplotlib widget

In [ ]:
output_dir = Path(r"C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos")
ROOT_DIR = Path(r"C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos")
rayleigh_fit_from_date(
    "ALHAMBRA",
    date(2023, 5, 10),
    initial_hour=2,
    duration=60,
    min_reference_height=6000,
    max_reference_height=7000,
    dir=ROOT_DIR,
    data_output_dir=output_dir,
    figure_output_dir=output_dir,
    save_fig=True,
    crop_ranges=(0,15000.)
)


In [ ]:
FLS2PLOT = [
    *Path(r"C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos\ALHAMBRA\QA\rayleigh_fit\2023\05\10").glob(
        "grRayleighFit*.nc"
    )
]
output_dir = Path(r"C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos")
plot_rayleigh_fit(
    FLS2PLOT,
    save_fig=True,
    output_dir=output_dir,
    range_limits=(0,15.)
)

In [ ]:
lidar = preprocess(r'C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos\ALHAMBRA\1a\2023\05\10\alh_1a_Prs_rs_xf_20230510.nc')
lidar.to_netcdf('preprocessed_alhambra_20230510.nc')

In [ ]:
lidar = preprocess(r'C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos\ALHAMBRA\1a\2023\05\09\alh_1a_Prs_rs_xf_20230509.nc')
lidar.to_netcdf('preprocessed_alhambra_20230509.nc')

In [ ]:
proc = xr.open_dataset(r'C:\Users\Usuario\Documents\gitlab\gfatpy\notebooks\preprocessed_alhambra_20230509.nc')

In [ ]:
(signal_to_rcs(proc.signal_532npa, proc.range)).plot(y='range',cmap='jet', vmin=0, vmax=1e6)

In [ ]:
from gfatpy.lidar.plot.quicklook import quicklook_xarray
fig, ax = quicklook_xarray(lidar.signal_532npa, is_rcs=False, scale_bounds=(4e5,8e5))
ax.set_ylim(0,1000)

In [ ]:
from gfatpy.lidar.plot.quicklook import quicklook_xarray
time_smooth = ("20230510T071500", "20230510T083600")
fig, ax = quicklook_xarray(lidar.signal_532npa.sel(time=slice(*time_smooth)), is_rcs=False, scale_bounds=(4e5,8e5))
ax.set_ylim(0,500)

In [ ]:
from gfatpy.lidar.plot.quicklook import quicklook_xarray
fig, ax = quicklook_xarray(lidar.signal_532npa, is_rcs=False, scale_bounds=(0,8e5))
ax.set_ylim(0,1000)

In [ ]:
from gfatpy.lidar.plot.quicklook import quicklook_xarray
fig, ax = quicklook_xarray(lidar.signal_355npa, is_rcs=False, scale_bounds=(0,4e6))
ax.set_ylim(0,1000)

In [ ]:
datestr, hour_ini, hour_end = '20230510', '050000', '060000' 
date_ini = datetime.strptime(datestr,'%Y%m%d')
date_ = date_ini
year, month, day = date_.year, date_.month, date_.day
ini_date_str = '%sT%s' % (datestr, hour_ini)
end_date_str = '%sT%s' % (datestr, hour_end)
fig, ax = plt.subplots(figsize=[10,5])
(lidar.signal_532npa.mean('time')/lidar.signal_532npa.sel(range=slice(2500,2600), time=slice(ini_date_str, end_date_str)).mean('time').mean('range')).plot(label='AN')
(lidar.signal_532npp.mean('time')/lidar.signal_532npp.sel(range=slice(2500,2600), time=slice(ini_date_str, end_date_str)).mean('time').mean('range')).plot(label='PC')
plt.xlim(0,3000)
plt.ylim(0.5,500)
plt.yscale('log')
plt.ylabel('normalized signal@532nm')
plt.legend()

In [ ]:
lidar['rcs_532npa'] = signal_to_rcs(lidar.signal_532npa, lidar.range)
lidar['rcs_532npp'] = signal_to_rcs(lidar.signal_532npp, lidar.range)
(lidar.rcs_532npa.mean('time')/lidar.rcs_532npa.sel(range=slice(2500,2600)).mean('time').mean('range')).plot(label='AN')
(lidar.rcs_532npp.mean('time')/lidar.rcs_532npp.sel(range=slice(2500,2600)).mean('time').mean('range')).plot(label='PC')

plt.xlim(0,1000)
plt.ylim(0,2)
plt.ylabel('normalized RCS@532nm')
plt.legend()

In [ ]:
time_smooth = ("20230510T073000", "20230510T083600")
fig, ax = plt.subplots(figsize=[5,7])
colors = color_list(len(lidar['signal_532npa'].sel(time=slice(*time_smooth)).time))
slidar = lidar['signal_532npa'].sel(time=slice(*time_smooth))
for idx, time_ in enumerate(slidar.time):
    signal_to_rcs(slidar, slidar.range).sel(time=time_).plot(y='range', c=colors[idx])
ax.set_ylim(100,500)
ax.set_xlim(5e5,1e6)
ax.set_xscale('log')
ax.set_xlabel('RCS@532nm')
plt.legend([])
ax.set_title(time_smooth)

In [ ]:
lidar['signal_532xsa'].mean('time').plot(x='range')
lidar['signal_532xpa'].mean('time').plot(x='range')
plt.ylim(-0.025,0.01)

In [ ]:
datestr, hour_ini, hour_end = '20230510', '050000', '060000' 
date_ini = datetime.strptime(datestr,'%Y%m%d')
date_ = date_ini
year, month, day = date_.year, date_.month, date_.day
ini_date_str = '%sT%s' % (datestr, hour_ini)
end_date_str = '%sT%s' % (datestr, hour_end)
rcs_532xpa = lidar['signal_532xpa']*lidar.range**2
rcs_532xpp = lidar['signal_532xpp']*lidar.range**2
rcs_532xsa = lidar['signal_532xsa']*lidar.range**2
rcs_532xsp = lidar['signal_532xsp']*lidar.range**2
rcs_532xpa_profile = rcs_532xpa.sel(time=slice(ini_date_str,end_date_str)).mean('time')
rcs_532xpp_profile = rcs_532xpp.sel(time=slice(ini_date_str,end_date_str)).mean('time')
rcs_532xsa_profile = rcs_532xsa.sel(time=slice(ini_date_str,end_date_str)).mean('time')
rcs_532xsp_profile = rcs_532xsp.sel(time=slice(ini_date_str,end_date_str)).mean('time')
nrcs_532xpa = rcs_532xpa_profile/rcs_532xpa_profile.sel(range=slice(8000,9000)).mean('range')
nrcs_532xpp = rcs_532xpp_profile/rcs_532xpp_profile.sel(range=slice(8000,9000)).mean('range')
nrcs_532xsa = rcs_532xsa_profile/np.abs(rcs_532xsa_profile.sel(range=slice(8000,9000)).mean('range'))
nrcs_532xsp = rcs_532xsp_profile/rcs_532xsp_profile.sel(range=slice(8000,9000)).mean('range')


In [ ]:
ranges = lidar.range
wavelength = 532
meteo_profiles = ecmwf.get_ecmwf_temperature_pressure(
    datestr, ranges_array=ranges
)


# Molecular Attenuated Backscatter
mol_properties = atmo.molecular_properties(
    wavelength, meteo_profiles['pressure'], meteo_profiles['temperature'], ranges
)
beta_mol_att = mol_properties["attenuated_molecular_beta"]


In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
nrcs_532xpa.plot(x='range', label='532xpa', ax=ax)
nrcs_532xpp.plot(x='range', label='532xpp', lw=4, ax=ax)
nrcs_532xsa.plot(x='range', label='532xsa', ax=ax)
nrcs_532xsp.plot(x='range', label='532xsp',lw=1, ax=ax)
(beta_mol_att/beta_mol_att.sel(range=slice(8000,9000)).mean('range')).plot(x='range',label='mol',color='k', ax=ax)
ax.legend()
ax.set_yscale('log')
ax.set_ylim(0.1,10)
# plt.savefig('rayleigh_gfatpy.png')

In [ ]:
import pickle
with open(r'C:\Users\Usuario\Documents\gitlab\gfatpy\licel_values.pkl', 'rb') as f:
    licel = pickle.load(f)    
with open(r'C:\Users\Usuario\Documents\gitlab\gfatpy\physical_values.pkl', 'rb') as f:
    physical = pickle.load(f)    

In [ ]:
(licel['BT11']/licel['BT11'].iloc[licel.index == 400].values.item()).plot()
(physical['BT11']/physical['BT11'].iloc[physical.index == 400].values.item()).plot(ls='--')
plt.xlim(0,1000)
plt.show()

In [ ]:
physical['BT11'].plot(ls='--')
plt.xlim(0,1000)
plt.show()

In [ ]:
filepath = r'C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos\ALHAMBRA\1a\2023\05\09\alh_1a_Prs_rs_xf_20230509.nc'
raw = xr.open_dataset(filepath)

In [ ]:
datestr, hour_ini, hour_end = '20230509', '020000', '030000' 
date_ini = datetime.strptime(datestr,'%Y%m%d')
date_ = date_ini
year, month, day = date_.year, date_.month, date_.day
ini_date_str = '%sT%s' % (datestr, hour_ini)
end_date_str = '%sT%s' % (datestr, hour_end)
raw.signal_532npa.sel(time=slice(ini_date_str, end_date_str)).mean('time').plot()
plt.xlim(0,1000)
plt.show()

In [ ]:
datestr, hour_ini, hour_end = '20230509', '020000', '030000' 
date_ini = datetime.strptime(datestr,'%Y%m%d')
date_ = date_ini
year, month, day = date_.year, date_.month, date_.day
ini_date_str = '%sT%s' % (datestr, hour_ini)
end_date_str = '%sT%s' % (datestr, hour_end)
raw.signal_355npa.sel(time=slice(ini_date_str, end_date_str)).mean('time').plot()
plt.xlim(0,1000)
plt.show()

In [ ]:
filepath = r'C:\Users\Usuario\Documents\gitlab\gfatpy\tests\datos\ALHAMBRA\1a\2023\05\09\alh_1a_Pdc_rs_xf_20230509_0100.nc'
dc = xr.open_dataset(filepath)

In [ ]:
dc

In [ ]:
datestr, hour_ini, hour_end = '20230509', '020000', '030000' 
date_ini = datetime.strptime(datestr,'%Y%m%d')
date_ = date_ini
year, month, day = date_.year, date_.month, date_.day
ini_date_str = '%sT%s' % (datestr, hour_ini)
end_date_str = '%sT%s' % (datestr, hour_end)
dc.signal_532npa.mean('time').plot()
plt.xlim(0,1000)
plt.show()

In [ ]:
dc.signal_532npa.sel(range=slice(0,5000)).std('range').std('time')

In [ ]:
raw.signal_532npa.sel(range=slice(0,5000)).std('range').std('time')